In [ ]:
# importing necessary libraries

import keras
import tensorflow
import numpy as np
from transformers import TFGPT2LMHeadModel
import pandas as pd
import numpy as np
import tensorflow as tf
import ast

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
# configuring tpu scope

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# loading the data

df=pd.read_csv('data_keywords.csv')

In [ ]:
!pip install transformers

In [ ]:
from transformers import GPT2Tokenizer

In [ ]:
# Initailizing the tokenizer

tokenizer=GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
# Adding Special Tokens to the GPT Model

tokenizer.add_special_tokens({
    'unk_token':'<unk>',
    'pad_token':'<pad>',
    'sep_token':'<sep>'
})

2

In [ ]:
# Adding a new token to the GPT Model

tokenizer.add_tokens(['<seperator>'])

1

In [ ]:
# reading our keywords and converging them to single string

def sep_keywords(x):
  x=ast.literal_eval(x)
  return ' <sep> '.join(x)

In [ ]:
# Applying the above function

df['keywords_with_sep']=df['keywords'].apply(sep_keywords)

In [ ]:
# So we use both texts

df['required_text']= df['keywords_with_sep']+' <seperator> '+ df['text']

In [ ]:
# Encoding the training data

res=tokenizer(df['required_text'].to_list(),padding=True,truncation=True)

In [ ]:
# Extract tokens from the encoding\

inputs=[]
attentions=[]
targets=[]
for i in range(38539):
  p=res['input_ids'][i]
  p1=res['attention_mask'][i]
  inputs.append(p[:-1])
  attentions.append(p1[:-1])
  targets.append(p[1:])

In [ ]:
del(res)

In [ ]:
# Converting lists to numpy arrays

inputs=np.array(inputs)
attentions=np.array(attentions)
targets=np.array(targets)

In [ ]:
# Transfer learning

with tpu_strategy.scope():
  model=TFGPT2LMHeadModel.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
# resizing the token embeddings of GPT-2

with tpu_strategy.scope():
  model.resize_token_embeddings(len(tokenizer))

In [ ]:
# creating the architecture 

with tpu_strategy.scope():
  inp1=Input((1023,),dtype='int32')
  inp2=Input((1023,),dtype='int32')
  emb=model(inp1,attention_mask=inp2)[0]
  m=Model(inputs=[inp1,inp2],outputs=emb)

In [ ]:
# compiling the model

with tpu_strategy.scope():
  m.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-5),loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
# training the model

m.fit([inputs,attentions],targets,epochs=3,batch_size=16,validation_split=0.15) 

Epoch 1/3
2048/2048 [==============================] - 563s 275ms/step - loss: 0.5752 - accuracy: 0.8828 - val_loss: 0.7655 - val_accuracy: 0.8562
Epoch 2/3
2048/2048 [==============================] - 562s 274ms/step - loss: 0.5683 - accuracy: 0.8836 - val_loss: 0.7650 - val_accuracy: 0.8564
Epoch 3/3
2048/2048 [==============================] - 562s 275ms/step - loss: 0.5601 - accuracy: 0.8846 - val_loss: 0.7663 - val_accuracy: 0.8563


In [ ]:
# saving the weights

save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
m.save_weights('/content/drive/MyDrive/gpt2 keyword model/gpt2 keyword model', options=save_locally)

In [ ]:
text = '''COVID-19 <sep> SARS-CoV-2 <sep> cardiac magnetic resonance <sep> chest x-ray <sep> computed tomography <sep> 
lung ultrasound <sep> echocardiography <seperator> The severe acute respiratory syndrome-coronavirus-2 outbreak has 
rapidly reached pandemic proportions and has become a major threat to global health.'''.lower()

input_ids = tokenizer.encode(text, return_tensors='tf')

beam_output = model.generate(
  input_ids,
  max_length = 200,
  num_beams=3
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [ ]:
res=tokenizer.decode(beam_output[0])

In [ ]:
p=res.split().index('<seperator>')
res=res.split()[p+1:]
p=res.index('<pad>')
res=res[:p]
res=' '.join(res)

## Generated Text


In [ ]:
res

'the severe acute respiratory syndrome-coronavirus-2 outbreak has rapidly reached pandemic proportions and has become a major threat to global health. chest x-ray (cxr) and computed tomography (ct) have been widely used in the diagnosis of covid-19. chest cxr and ct have been widely used in the diagnosis of sars-cov-2 infection, but there is a lack of consensus regarding the clinical utility of cxr and ct in the diagnosis of covid-19. the purpose of this study was to evaluate the clinical utility of chest cxr and ct in the diagnosis of covid-19.'

So we have the generated text by the model, which is very similar to the keywords we inputted the model with. So the model is contextually able to identify the meaning of keywords